In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Training, Tuning and Deploying a PyTorch Text Classification Model on [Vertex AI](https://cloud.google.com/vertex-ai)
## Fine-tuning pre-trained [BERT](https://huggingface.co/bert-base-cased) model for sentiment classification task 

# Overview

This example is inspired from Token-Classification [notebook](https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb) and [run_glue.py](https://github.com/huggingface/transformers/blob/v2.5.0/examples/run_glue.py). 
We will be fine-tuning **`bert-base-cased`** (pre-trained) model for sentiment classification task.
You can find the details about this model at [Hugging Face Hub](https://huggingface.co/bert-base-cased).

For more notebooks with the state of the art PyTorch/Tensorflow/JAX, you can explore [Hugging FaceNotebooks](https://huggingface.co/transformers/notebooks.html).

### Dataset

We will be using [IMDB movie review dataset](https://huggingface.co/datasets/imdb) from [Hugging Face Datasets](https://huggingface.co/datasets).

### Objective

How to **Build, Train, Tune and Deploy PyTorch models on [Vertex AI](https://cloud.google.com/vertex-ai)** and emphasize first class support for training and deploying PyTorch models on Vertex AI. 

### Table of Contents

This notebook covers following sections:

- [Creating Notebooks instance](#Creating-Notebooks-instance-on-Google-Cloud)
- [Training](#Training)
    - [Run Training Locally in the Notebook](#Training-locally-in-the-notebook)
    - [Run Training Job on Vertex AI](#Training-on-Vertex-AI)
        - [Training with pre-built container](#Run-Custom-Job-on-Vertex-AI-Training-with-a-pre-built-container)
        - [Training with custom container](#Run-Custom-Job-on-Vertex-AI-Training-with-custom-container)
- [Tuning](#Hyperparameter-Tuning) 
    - [Run Hyperparameter Tuning job on Vertex AI](#Run-Hyperparameter-Tuning-Job-on-Vertex-AI)
- [Deploying](#Deploying)
    - [Deploying model on Vertex AI Predictions with custom container](#Deploying-model-on-Vertex AI-Predictions-with-custom-container)

### Install additional packages

Python dependencies required for this notebook are [Transformers](https://pypi.org/project/transformers/), [Datasets](https://pypi.org/project/datasets/) and [hypertune](https://github.com/GoogleCloudPlatform/cloudml-hypertune) will be installed in the Notebooks instance itself.

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
!pip -q install {USER_FLAG} --upgrade transformers
!pip -q install {USER_FLAG} --upgrade datasets
!pip -q install {USER_FLAG} --upgrade tqdm
!pip -q install {USER_FLAG} --upgrade cloudml-hypertune

We will be using [Vertex AI SDK for Python](https://cloud.google.com/vertex-ai/docs/start/client-libraries#python) to interact with Vertex AI services. The high-level `aiplatform` library is designed to simplify common data science workflows by using wrapper classes and opinionated defaults. 

#### Install Vertex AI SDK for Python

In [ ]:
!pip -q install {USER_FLAG} --upgrade google-cloud-aiplatform

### Restart the Kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

## Before you begin

### Select a GPU runtime

**Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select "Runtime --> Change runtime type > GPU"**

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.
1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).
1. Enable following APIs in your project required for running the tutorial
    - [Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)
    - [Cloud Storage API](https://console.cloud.google.com/flows/enableapi?apiid=storage.googleapis.com)
    - [Container Registry API](https://console.cloud.google.com/flows/enableapi?apiid=containerregistry.googleapis.com)
    - [Cloud Build API](https://console.cloud.google.com/flows/enableapi?apiid=cloudbuild.googleapis.com)
1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).
1. Enter your project ID in the cell below. Then run the cell to make sure the Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud` or `google.auth`.

In [ ]:
PROJECT_ID = "qwiklabs-gcp-04-2329cec2130c"  # <--- #TODO 1 CHANGE THIS TO YOUR PROJECT

import os

# Get your Google Cloud project ID using google.auth
if not os.getenv("IS_TESTING"):
    import google.auth

    _, PROJECT_ID = google.auth.default()
    print("Project ID: ", PROJECT_ID)

# validate PROJECT_ID
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    print(
        f"Please set your project id before proceeding to next step. Currently it's set as {PROJECT_ID}"
    )

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime


def get_timestamp():
    return datetime.now().strftime("%Y%m%d%H%M%S")


TIMESTAMP = get_timestamp()
print(f"TIMESTAMP = {TIMESTAMP}")

### Create a Cloud Storage bucket


When you submit a training job using the Cloud SDK, you upload a Python package containing your training code to a Cloud Storage bucket. Vertex AI runs the code from this package. In this tutorial, Vertex AI also saves the trained model that results from your job in the same bucket. Using this model artifact, you can then create Vertex AI model and endpoint resources in order to serve online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may not use a Multi-Regional Storage bucket for training with Vertex AI.

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # Keep as is
REGION = "us-central1"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = f"gs://{PROJECT_ID}aip-{get_timestamp()}"

In [ ]:
print(f"PROJECT_ID = {PROJECT_ID}")
print(f"BUCKET_NAME = {BUCKET_NAME}")
print(f"REGION = {REGION}")

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME  ## If it runs succesfully you will see no errors. if you just created, no files will be there

### Import libraries and define constants

In [ ]:
import base64
import json
import os
import random
import sys

import google.auth
from google.cloud import aiplatform
from google.cloud.aiplatform import gapic as aip
from google.cloud.aiplatform import hyperparameter_tuning as hpt
from google.protobuf.json_format import MessageToDict

In [ ]:
from IPython.display import HTML, display

In [ ]:
import datasets
import numpy as np
import pandas as pd
import torch
import transformers
from datasets import ClassLabel, Sequence, load_dataset
from transformers import (AutoModelForSequenceClassification, AutoTokenizer,
                          EvalPrediction, Trainer, TrainingArguments,
                          default_data_collator)

In [ ]:
print(f"Notebook runtime: {'GPU' if torch.cuda.is_available() else 'CPU'}")
print(f"PyTorch version : {torch.__version__}")
print(f"Datasets version : {datasets.__version__}")
print(f"Transformers version : {transformers.__version__}")

In [ ]:
APP_NAME = "finetuned-bert-classifier"

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Training

In this section, we will train a PyTorch model by fine-tuning pre-trained model from [Hugging Face Transformers](https://github.com/huggingface/transformers).

## Training locally in the notebook

### Loading the dataset

For this example we will use [IMDB movie review dataset](https://huggingface.co/datasets/imdb) from [Hugging Face Datasets](https://huggingface.co/datasets/) for sentiment classification task. We use the [Hugging Face Datasets](https://github.com/huggingface/datasets) library to download the data. This can be easily done with the function `load_dataset`.


In [ ]:
datasets = load_dataset("imdb")
datasets

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [ ]:
print(
    "Total # of rows in training dataset {} and size {:5.2f} MB".format(
        datasets["train"].shape[0], datasets["train"].size_in_bytes / (1024 * 1024)
    )
)
print(
    "Total # of rows in test dataset {} and size {:5.2f} MB".format(
        datasets["test"].shape[0], datasets["test"].size_in_bytes / (1024 * 1024)
    )
)

To access an actual element, you need to select a split first, then give an index:

In [ ]:
datasets["train"][0]

Using the `unique` method to extract label list. This will allow us to experiment with other datasets without hard-coding labels.

In [ ]:
label_list = datasets["train"].unique("label")
label_list

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [ ]:
def show_random_elements(dataset, num_examples=2):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

### Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a pre-trained Hugging Face Transformers [`Tokenizer` class](https://huggingface.co/transformers/main_classes/tokenizer.html) which tokenizes the inputs (including converting the tokens to their corresponding IDs in the pre-trained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which ensures:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pre-training this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
batch_size = 16
max_seq_length = 128
model_name_or_path = "bert-base-cased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    use_fast=True,
)
# 'use_fast' ensure that we use fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can check type of models available with a fast tokenizer on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [ ]:
tokenizer("Hello, this is one sentence!")

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

**NOTE:** If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenizer with the argument `is_split_into_words=True`:

In [ ]:
example = datasets["train"][4]
print(example)

In [ ]:
tokenizer(
    ["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."],
    is_split_into_words=True,
)

Note that transformers are often pre-trained with sub-word tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [ ]:
# Dataset loading repeated here to make this cell idempotent
# Since we are over-writing datasets variable
datasets = load_dataset("imdb")

# Mapping labels to ids
# NOTE: We can extract this automatically but the `Unique` method of the datasets
# is not reporting the label -1 which shows up in the pre-processing.
# Hence the additional -1 term in the dictionary
label_to_id = {1: 1, 0: 0, -1: 0}


def preprocess_function(examples):
    """
    Tokenize the input example texts
    NOTE: The same preprocessing step(s) will be applied
    at the time of inference as well.
    """
    args = (examples["text"],)
    result = tokenizer(
        *args, padding="max_length", max_length=max_seq_length, truncation=True
    )

    # Map labels to IDs (not necessary for GLUE tasks)
    if label_to_id is not None and "label" in examples:
        result["label"] = [label_to_id[example] for example in examples["label"]]

    return result


# apply preprocessing function to input examples
datasets = datasets.map(preprocess_function, batched=True, load_from_cache_file=True)

### Fine-tuning the model

Now that our data is ready, we can download the pre-trained model and fine-tune it.

---

***Fine Tuning*** *involves taking a model that has already been trained for a given task and then tweaking the model for another similar task. Specifically, the tweaking involves replicating all the layers in the pre-trained model including weights and parameters, except the output layer. Then adding a new output classifier layer that predicts labels for the current task. The final step is to train the output layer from scratch, while the parameters of all layers from the pre-trained model are frozen. This allows learning from the pre-trained representations and "fine-tuning" the higher-order feature representations more relevant for the concrete task, such as analyzing sentiments in this case.* 

*For the scenario in the notebook analyzing sentiments, the pre-trained BERT model already encodes a lot of information about the language as the model was trained on a large corpus of English data in a self-supervised fashion. Now we only need to slightly tune them using their outputs as features for the sentiment classification task. This means quicker development iteration on a much smaller dataset, instead of training a specific Natural Language Processing (NLP) model with a larger training dataset.*


![BERT fine-tuning](./images/bert-model-fine-tuning.png)

---


Since all our tasks are about token classification, we use the `AutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name_or_path, num_labels=len(label_list)
)

**NOTE:** The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pre-train the model on a masked language modeling objective and replacing it with a new head for which we don't have pre-trained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
args = TrainingArguments(
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    output_dir="/tmp/cls",
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. You can define your custom compute_metrics function. It takes an `EvalPrediction` object (a namedtuple with a predictions and label_ids field) and has to return a dictionary string to float.

In [ ]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

Now we create the `Trainer` object and we are almost ready to train.

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["test"],
    data_collator=default_data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

You can add callbacks to the `trainer` object to customize the behavior of the training loop such as early stopping, reporting metrics at the end of evaluation phase or taking any decisions. In the hyperparameter tuning section of this notebook, we add a callback to `trainer` for automating hyperparameter tuning process.

We can now fine-tune our model by just calling the `train` method:

In [ ]:
trainer.train()

In [ ]:
saved_model_local_path = "./models"
!mkdir ./models

In [ ]:
trainer.save_model(saved_model_local_path)

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
history = trainer.evaluate()

In [ ]:
history

To get the other metrics computed  such as precision, recall or F1 score for each category, we can apply the same function as before on the result of the `predict` method.

### Run predictions locally with sample examples

Using the trained model, we can predict the sentiment label for an input text after applying the preprocessing function that was used during the training. We will run the predictions locally in the notebook and later show how you can deploy the model to an endpoint using [TorchServe](https://pytorch.org/serve/) on Vertex AI Predictions.

In [ ]:
model_name_or_path = "bert-base-cased"
label_text = {0: "Negative", 1: "Positive"}
saved_model_path = saved_model_local_path


def predict(input_text, saved_model_path):
    # initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

    # preprocess and encode input text
    tokenizer_args = (input_text,)
    predict_input = tokenizer(
        *tokenizer_args,
        padding="max_length",
        max_length=128,
        truncation=True,
        return_tensors="pt",
    )

    # load trained model
    loaded_model = AutoModelForSequenceClassification.from_pretrained(saved_model_path)

    # get predictions
    output = loaded_model(predict_input["input_ids"])

    # return labels
    label_id = torch.argmax(*output.to_tuple(), dim=1)

    print(f"Review text: {input_text}")
    print(f"Sentiment : {label_text[label_id.item()]}\n")

In [ ]:
# example #1
review_text = (
    """Jaw dropping visual affects and action! One of the best I have seen to date."""
)
predict_input = predict(review_text, saved_model_path)

In [ ]:
# example #2
review_text = """Take away the CGI and the A-list cast and you end up with film with less punch."""
predict_input = predict(review_text, saved_model_path)

Validate the model artifacts written to GCS by the training code after the job completes successfully

#### **Create a custom model handler to handle prediction requests**

When predicting sentiments of the input text with the fine-tuned transformer model, it requires pre-processing of the input text and post-processing by adding name (positive/negative) to the target label (1/0) along with probability (or confidence). We create a custom handler script that is packaged with the model artifacts and TorchServe executes the code when it runs. 

Custom handler script does the following:

- Pre-process input text before sending it to the model for inference
- Customize how the model is invoked for inference
- Post-process output from the model before sending back a response

Please refer to the [TorchServe documentation](https://pytorch.org/serve/custom_service.html) for defining a custom handler.

In [ ]:
%%writefile models/custom_handler.py

import os
import json
import logging

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from ts.torch_handler.base_handler import BaseHandler

logger = logging.getLogger(__name__)


class TransformersClassifierHandler(BaseHandler):
    """
    The handler takes an input string and returns the classification text 
    based on the serialized transformers checkpoint.
    """
    def __init__(self):
        super(TransformersClassifierHandler, self).__init__()
        self.initialized = False

    def initialize(self, ctx):
        """ Loads the model.pt file and initialized the model object.
        Instantiates Tokenizer for preprocessor to use
        Loads labels to name mapping file for post-processing inference response
        """
        self.manifest = ctx.manifest

        properties = ctx.system_properties
        model_dir = properties.get("model_dir")
        self.device = torch.device("cuda:" + str(properties.get("gpu_id")) if torch.cuda.is_available() else "cpu")

        # Read model serialize/pt file
        serialized_file = self.manifest["model"]["serializedFile"]
        model_pt_path = os.path.join(model_dir, serialized_file)
        if not os.path.isfile(model_pt_path):
            raise RuntimeError("Missing the model.pt or pytorch_model.bin file")
        
        # Load model
        self.model = AutoModelForSequenceClassification.from_pretrained(model_dir)
        self.model.to(self.device)
        self.model.eval()
        logger.debug('Transformer model from path {0} loaded successfully'.format(model_dir))
        
        # Ensure to use the same tokenizer used during training
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

        # Read the mapping file, index to object name
        mapping_file_path = os.path.join(model_dir, "index_to_name.json")

        if os.path.isfile(mapping_file_path):
            with open(mapping_file_path) as f:
                self.mapping = json.load(f)
        else:
            logger.warning('Missing the index_to_name.json file. Inference output will default.')
            self.mapping = {"0": "Negative",  "1": "Positive"}

        self.initialized = True

    def preprocess(self, data):
        """ Preprocessing input request by tokenizing
            Extend with your own preprocessing steps as needed
        """
        text = data[0].get("data")
        if text is None:
            text = data[0].get("body")
        sentences = text.decode('utf-8')
        logger.info("Received text: '%s'", sentences)

        # Tokenize the texts
        tokenizer_args = ((sentences,))
        inputs = self.tokenizer(*tokenizer_args,
                                padding='max_length',
                                max_length=128,
                                truncation=True,
                                return_tensors = "pt")
        return inputs

    def inference(self, inputs):
        """ Predict the class of a text using a trained transformer model.
        """
        prediction = self.model(inputs['input_ids'].to(self.device))[0].argmax().item()

        if self.mapping:
            prediction = self.mapping[str(prediction)]

        logger.info("Model predicted: '%s'", prediction)
        return [prediction]

    def postprocess(self, inference_output):
        return inference_output


##### **Generate target label to name file** *[Optional]*

In the custom handler, we refer to a mapping file between target labels and their meaningful names that will be used to format the prediction response. Here we are mapping target label "0" as "Negative" and "1"  as "Positive". 

In [ ]:
%%writefile ./models/index_to_name.json

{
    "0": "Negative", 
    "1": "Positive"
}

#### **Create custom container image to serve predictions**

We will use Cloud Build to create the custom container image with following build steps:

##### **Download model artifacts**

Download model artifacts that were saved as part of the training (or hyperparameter tuning) job from Cloud Storage to local directory

##### **Build the container image**

Create a Dockerfile with TorchServe as base image:

 - **`RUN`**: Installs dependencies such as `transformers`
 - **`COPY`**: Add model artifacts to `/home/model-server/` directory of the container image
 - **`COPY`**: Add custom handler script to `/home/model-server/` directory of the container image
 - **`RUN`**: Create `/home/model-server/config.properties` to define the serving configuration (health and prediction listener ports)
 - **`RUN`**: Run [Torch model archiver](https://pytorch.org/serve/model-archiver.html) to create a model archive file from the files copied into the image `/home/model-server/`. The model archive is saved in the `/home/model-server/model-store/` with name same as `<model-name>.mar`
 - **`CMD`**: Launch Torchserve HTTP server referencing the configuration properties and enables serving for the model

In [ ]:
mkdir -p models/model/finetuned-bert-classifier

In [ ]:
cp models/*.* models/model/finetuned-bert-classifier/

In [ ]:
%%bash -s $APP_NAME

APP_NAME=$1

cat << EOF > ./models/Dockerfile

FROM pytorch/torchserve:latest-cpu

# install dependencies
RUN python3 -m pip install --upgrade pip
RUN pip3 install transformers

USER model-server

# copy model artifacts, custom handler and other dependencies
COPY ./custom_handler.py /home/model-server/
COPY ./index_to_name.json /home/model-server/
COPY ./model/$APP_NAME/ /home/model-server/

# create torchserve configuration file
USER root
RUN printf "\nservice_envelope=json" >> /home/model-server/config.properties
RUN printf "\ninference_address=http://0.0.0.0:7080" >> /home/model-server/config.properties
RUN printf "\nmanagement_address=http://0.0.0.0:7081" >> /home/model-server/config.properties
USER model-server

# expose health and prediction listener ports from the image
EXPOSE 7080
EXPOSE 7081

# create model archive file packaging model artifacts and dependencies
RUN torch-model-archiver -f \
  --model-name=$APP_NAME \
  --version=1.0 \
  --serialized-file=/home/model-server/pytorch_model.bin \
  --handler=/home/model-server/custom_handler.py \
  --extra-files "/home/model-server/config.json,/home/model-server/tokenizer.json,/home/model-server/training_args.bin,/home/model-server/tokenizer_config.json,/home/model-server/special_tokens_map.json,/home/model-server/vocab.txt,/home/model-server/index_to_name.json" \
  --export-path=/home/model-server/model-store

# run Torchserve HTTP serve to respond to prediction requests
CMD ["torchserve", \
     "--start", \
     "--ts-config=/home/model-server/config.properties", \
     "--models", \
     "$APP_NAME=$APP_NAME.mar", \
     "--model-store", \
     "/home/model-server/model-store"]
EOF

echo "Writing ./models/Dockerfile"

Build the docker image tagged with Container Registry (gcr.io) path

In [ ]:
CUSTOM_PREDICTOR_IMAGE_URI = f"gcr.io/{PROJECT_ID}/pytorch_predict_{APP_NAME}"
print(f"CUSTOM_PREDICTOR_IMAGE_URI = {CUSTOM_PREDICTOR_IMAGE_URI}")

In [ ]:
!docker build \
  --tag=$CUSTOM_PREDICTOR_IMAGE_URI\
./models

#### **Run the container locally** *** 

Before push the container image to Container Registry to use it with Vertex AI Predictions, you can run it as a container in your local environment to verify that the server works as expected

1. To run the container image as a container locally, run the following command:

In [ ]:
!docker stop local_bert_classifier
!docker run -t -d --rm -p 7080:7080 --name=local_bert_classifier $CUSTOM_PREDICTOR_IMAGE_URI
!sleep 20

2. To send the container's server a health check, run the following command:

In [ ]:
!curl http://localhost:7080/ping

If successful, the server returns the following response:

```
{
  "status": "Healthy"
}
```

3. To send the container's server a prediction request, run the following commands:

In [ ]:
%%bash -s $APP_NAME

APP_NAME=$1

cat > ./predictor/instances.json <<END
{ 
   "instances": [
     { 
       "data": {
         "b64": "$(echo 'Take away the CGI and the A-list cast and you end up with film with less punch.' | base64 --wrap=0)"
       }
     }
   ]
}
END

curl -s -X POST \
  -H "Content-Type: application/json; charset=utf-8" \
  -d @./predictor/instances.json \
  http://localhost:7080/predictions/$APP_NAME/

This request uses a test sentence. If successful, the server returns prediction in below format:

```
    {"predictions": ["Negative"]}
```

4. To stop the container, run the following command:

In [ ]:
!docker stop local_bert_classifier

### Deploying the model to a simulated edge device (linux vm with docker runtime) [Optional]

##### **Push the serving container to Container Registry**

Push your container image with inference code and dependencies to your Container Registry

In [ ]:
!docker push $CUSTOM_PREDICTOR_IMAGE_URI

### Create a Linux VM with docker runtime

1. Copy/Run the entire following script in Cloud Shell:

gcloud compute instances create docker-vm  \
--zone=us-central1-a \
--image=ubuntu-1804-bionic-v20220213 \
--image-project=ubuntu-os-cloud \
 --machine-type=e2-small \
 --metadata=startup-script='#! /bin/bash

sudo apt-get update
sudo apt-get -y install \
    ca-certificates \
    curl \
    gnupg \
    lsb-release

curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg

echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu \
  $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null

sudo apt-get update

sudo apt-get install -y docker-ce docker-ce-cli containerd.io'

In [ ]:
#login via ssh in cloud shell
gcloud compute ssh docker-vm --zone=us-central1-a

In [ ]:
#run these gcloud commands in ssh terminal to authenticate your ID
#You'll be asked to go to a webpage to authenticate your google id
gcloud auth login

gcloud auth configure-docker

In [ ]:
#Run these commands to check if docker is installed and you have access to Google Cloud Registry (gcr)
sudo docker run hello-world
sudo docker pull gcr.io/cloud-marketplace/google/nginx1:latest

In [ ]:
####Continue from above run the following via SSH to pull in the docker image for the model and run predictions against it
####TODO Replace qwiklabs-gcp-04-2329cec2130c with your project ID 

sudo docker pull gcr.io/qwiklabs-gcp-04-2329cec2130c/pytorch_predict_finetuned-bert-classifier

sudo docker run -t -d --rm -p 7080:7080 --name=local_bert_classifier gcr.io/qwiklabs-gcp-04-2329cec2130c/

curl http://localhost:7080/ping

cat > instances.json <<END
> { 
>    "instances": [
>      { 
>        "data": {
>          "b64": "$(echo 'Take away the CGI and the A-list cast and you end up with film with less punch.' | base64 --wrap=0)"
>        }
>      }
>    ]
> }
> END


## this is getting an actual prediction from the model
curl -s -X POST \
>   -H "Content-Type: application/json; charset=utf-8" \
>   -d @./instances.json \
>   http://localhost:7080/predictions/finetuned-bert-classifier/

#### **Deploying the serving container to Vertex AI Predictions[Optional]**

We create a model resource on Vertex AI and deploy the model to a Vertex AI Endpoints. You must deploy a model to an endpoint before using the model. The deployed model runs the custom container image to serve predictions. 

##### **Initialize the Vertex AI SDK for Python**

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

##### **Create a Model resource with custom serving container**

In [ ]:
VERSION = 1
model_display_name = f"{APP_NAME}-v{VERSION}"
model_description = "PyTorch based text classifier with custom container"

MODEL_NAME = APP_NAME
health_route = "/ping"
predict_route = f"/predictions/{MODEL_NAME}"
serving_container_ports = [7080]

In [ ]:
model = aiplatform.Model.upload(
    display_name=model_display_name,
    description=model_description,
    serving_container_image_uri=CUSTOM_PREDICTOR_IMAGE_URI,
    serving_container_predict_route=predict_route,
    serving_container_health_route=health_route,
    serving_container_ports=serving_container_ports,
)

model.wait()

print(model.display_name)
print(model.resource_name)

For more context on upload or importing a model, refer [documentation](https://cloud.google.com/vertex-ai/docs/general/import-model)

##### **Create an Endpoint for Model with Custom Container**

In [ ]:
endpoint_display_name = f"{APP_NAME}-endpoint"
endpoint = aiplatform.Endpoint.create(display_name=endpoint_display_name)

##### **Deploy the Model to Endpoint**

Deploying a model associates physical resources with the model so it can serve online predictions with low latency. 

**NOTE:** This step takes few minutes to deploy the resources.

In [ ]:
traffic_percentage = 100
machine_type = "n1-standard-4"
deployed_model_display_name = model_display_name
min_replica_count = 1
max_replica_count = 3
sync = True

model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=deployed_model_display_name,
    machine_type=machine_type,
    traffic_percentage=traffic_percentage,
    sync=sync,
)

#### **Invoking the Endpoint with deployed Model using Vertex AI SDK to make predictions**

##### **Get the Endpoint id**

In [ ]:
endpoint_display_name = f"{APP_NAME}-endpoint"
filter = f'display_name="{endpoint_display_name}"'

for endpoint_info in aiplatform.Endpoint.list(filter=filter):
    print(
        f"Endpoint display name = {endpoint_info.display_name} resource id ={endpoint_info.resource_name} "
    )

endpoint = aiplatform.Endpoint(endpoint_info.resource_name)

In [ ]:
endpoint.list_models()

##### **Formatting input for online prediction**

This notebook uses [Torchserve's KServe based inference API](https://pytorch.org/serve/inference_api.html#kserve-inference-api) which is also [Vertex AI Predictions compatible format](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements#prediction). For online prediction requests, format the prediction input instances as JSON with base64 encoding as shown here:

```
[
    {
        "data": {
            "b64": "<base64 encoded string>"
        }
    }
]
```

Define sample texts to test predictions

In [ ]:
test_instances = [
    b"Jaw dropping visual affects and action! One of the best I have seen to date.",
    b"Take away the CGI and the A-list cast and you end up with film with less punch.",
]

##### **Sending an online prediction request**

Format input text string and call prediction endpoint with formatted input request and get the response

In [ ]:
print("=" * 100)
for instance in test_instances:
    print(f"Input text: \n\t{instance.decode('utf-8')}\n")
    b64_encoded = base64.b64encode(instance)
    test_instance = [{"data": {"b64": f"{str(b64_encoded.decode('utf-8'))}"}}]
    print(f"Formatted input: \n{json.dumps(test_instance, indent=4)}\n")
    prediction = endpoint.predict(instances=test_instance)
    print(f"Prediction response: \n\t{prediction}")
    print("=" * 100)